In [1]:
import torch
import torch.nn as nn
from transformers import GPT2Model
import torch.nn.functional as F
import wandb
from tqdm import tqdm

import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from einops import rearrange

In [2]:
class GPT2CIFAR10(nn.Module):
    def __init__(self, patch_size=4, num_classes=10, freeze_gpt2=True):
        super().__init__()
        
        # Load pretrained GPT2
        self.gpt2 = GPT2Model.from_pretrained('gpt2')
        self.hidden_size = self.gpt2.config.hidden_size  # 768 for base GPT2
        
        # CIFAR-10 characteristics
        self.image_size = 32
        self.patch_size = patch_size
        self.num_patches = (self.image_size // patch_size) ** 2
        
        # Patch embedding layer: from image patches to GPT2 hidden size
        self.patch_embedding = nn.Conv2d(3, self.hidden_size, 
                                       kernel_size=patch_size, 
                                       stride=patch_size)
        
        # Classification head
        self.classifier = nn.Linear(self.hidden_size, num_classes)
        
        if freeze_gpt2:
            # Freeze GPT2 parameters except LayerNorm and positional embeddings
            for name, param in self.gpt2.named_parameters():
                if 'ln' in name or 'wpe' in name:
                    param.requires_grad = True
                else:
                    param.requires_grad = False
    
    def forward(self, x):
        batch_size = x.shape[0]
        
        # Convert image to patches
        # Shape: (batch_size, hidden_size, h', w')
        patches = self.patch_embedding(x)
        
        # Reshape and transpose for GPT2
        # Shape: (batch_size, num_patches, hidden_size)
        patches = rearrange(patches, 'b d h w -> b (h w) d')
        
        # Pass through GPT2 and get last hidden state
        outputs = self.gpt2(inputs_embeds=patches)
        hidden_states = outputs.last_hidden_state
        
        # Use the last token's representation for classification
        cls_representation = hidden_states[:, -1]
        
        # Classify
        logits = self.classifier(cls_representation)
        
        return logits

In [3]:
class CIFAR10Trainer:
    def __init__(self, model, train_loader, val_loader, 
                 criterion, optimizer, device, config):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = device
        self.config = config
        
        # Initialize metrics tracking
        self.train_losses = []
        self.train_accuracies = []
        self.val_losses = []
        self.val_accuracies = []
        
        # Initialize best validation accuracy for model saving
        self.best_val_acc = 0.0
        
    def train_epoch(self):
        self.model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        pbar = tqdm(self.train_loader, desc='Training')
        for batch_idx, (images, labels) in enumerate(pbar):
            images, labels = images.to(self.device), labels.to(self.device)
            
            self.optimizer.zero_grad()
            outputs = self.model(images)
            loss = self.criterion(outputs, labels)
            
            loss.backward()
            self.optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # Update progress bar
            pbar.set_postfix({
                'loss': running_loss/(batch_idx+1),
                'acc': 100.*correct/total
            })
        
        epoch_loss = running_loss / len(self.train_loader)
        epoch_acc = 100. * correct / total
        return epoch_loss, epoch_acc
    
    def validate(self):
        self.model.eval()
        running_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in tqdm(self.val_loader, desc='Validation'):
                images, labels = images.to(self.device), labels.to(self.device)
                
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                
                running_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
        
        val_loss = running_loss / len(self.val_loader)
        val_acc = 100. * correct / total
        return val_loss, val_acc
    
    def train(self, num_epochs):
        # Initialize wandb
        wandb.init(
            project="gpt2-cifar10", 
            config=self.config,
            name=f'{self.config["learning_rate"]}lr_{self.config["batch_size"]}bs_{self.config["patch_size"]}patch'
        )
        
        for epoch in range(num_epochs):
            # Training phase
            train_loss, train_acc = self.train_epoch()
            self.train_losses.append(train_loss)
            self.train_accuracies.append(train_acc)
            
            # Validation phase
            val_loss, val_acc = self.validate()
            self.val_losses.append(val_loss)
            self.val_accuracies.append(val_acc)
            
            # Log metrics
            wandb.log({
                "epoch": epoch,
                "train_loss": train_loss,
                "train_acc": train_acc,
                "val_loss": val_loss,
                "val_acc": val_acc,
            })
            
            # Save best model
            if val_acc > self.best_val_acc:
                self.best_val_acc = val_acc
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'val_acc': val_acc,
                }, 'best_model.pth')
                wandb.save('best_model.pth')
            
            print(f'Epoch: {epoch+1}/{num_epochs}')
            print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
            print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
            print('-' * 60)
        
        wandb.finish()

In [4]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load CIFAR10
trainset = datasets.CIFAR10(root='./data', train=True,
                          download=True, transform=transform_train)
valset = datasets.CIFAR10(root='./data', train=False,
                         download=True, transform=transform_val)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(valset, batch_size=128, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:07<00:00, 23.3MB/s] 


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
# Configuration
config = {
    'patch_size': 4,
    'num_classes': 10,
    'batch_size': 16,
    'learning_rate': 1e-3,
    'num_epochs': 100,
}

# Initialize model and training components
model = GPT2CIFAR10(patch_size=config['patch_size'], 
                    num_classes=config['num_classes'])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Initialize trainer
trainer = CIFAR10Trainer(
    model=model,
    train_loader=trainloader,
    val_loader=valloader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    config=config
)

# Train model
trainer.train(num_epochs=config['num_epochs'])

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dporres (dporres-computer-vision-center). Use `wandb login --relogin` to force relogin


Validation: 100%|██████████| 79/79 [00:17<00:00,  4.57it/s]


Epoch: 1/50
Train Loss: 2.3167, Train Acc: 14.61%
Val Loss: 2.1477, Val Acc: 21.83%
------------------------------------------------------------


Validation: 100%|██████████| 79/79 [00:16<00:00,  4.70it/s]


Epoch: 2/50
Train Loss: 2.1854, Train Acc: 19.93%
Val Loss: 2.0410, Val Acc: 24.67%
------------------------------------------------------------


...

Validation: 100%|██████████| 79/79 [00:17<00:00,  4.63it/s]


Epoch: 49/50
Train Loss: 1.1275, Train Acc: 59.52%
Val Loss: 1.0723, Val Acc: 61.98%
------------------------------------------------------------


Validation: 100%|██████████| 79/79 [00:16<00:00,  4.68it/s]


Epoch: 50/50
Train Loss: 1.1245, Train Acc: 59.68%
Val Loss: 1.0737, Val Acc: 62.22%
------------------------------------------------------------


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇███
train_acc,▁▂▃▃▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████
train_loss,█▇▆▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▁▂▃▃▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██████████████
val_loss,█▇▇▆▆▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,49
train_acc,59.676
train_loss,1.12449
val_acc,62.22
val_loss,1.0737


## Visualizing trained model

In [6]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np

from PIL import Image

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
class GPT2Visualizer:
    def __init__(self, model, device, class_names):
        self.model = model.to(device)
        self.device = device
        self.class_names = class_names
        self.model.eval()
        
        # Save reference to GPT2 attention
        self.attention_maps = []
        
        # Register hook to get attention weights
        def attention_hook(module, input, output):
            # Get attention weights from output tuple
            # Shape: (batch_size, num_heads, sequence_length, sequence_length)
            self.attention_maps.append(output[0].detach())
        
        # Register hooks for all attention blocks
        for name, module in model.named_modules():
            if "attn" in name and "block" in name:
                module.register_forward_hook(attention_hook)
        
        # Standard CIFAR-10 normalization
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), 
                               (0.2023, 0.1994, 0.2010))
        ])
    
    def predict_and_visualize(self, images, true_labels=None, num_images=5):
        """
        Visualize predictions and attention maps for a batch of images
        
        Args:
            images: List of PIL images or tensor of shape (N, C, H, W)
            true_labels: Optional list of true labels
            num_images: Number of images to visualize
        """
        # Clear previous attention maps
        self.attention_maps = []
        
        # Prepare images if they're PIL
        if not torch.is_tensor(images):
            tensors = []
            for img in images:
                tensors.append(self.transform(img))
            images = torch.stack(tensors)
        
        # Move to device
        images = images.to(self.device)
        
        # Get predictions
        with torch.no_grad():
            outputs = self.model(images[:num_images])
            predictions = outputs.argmax(dim=1)
        
        # Get attention weights (average over heads and layers)
        # Shape: (batch_size, num_patches, num_patches)
        avg_attention = torch.mean(torch.stack([
            torch.mean(attention, dim=1) 
            for attention in self.attention_maps
        ]), dim=0)
        
        # Create figure
        num_cols = 3  # image, attention, patch attention
        fig = plt.figure(figsize=(15, 5 * num_images))
        
        for idx in range(num_images):
            # Original image with prediction
            ax1 = plt.subplot(num_images, num_cols, idx * num_cols + 1)
            img = images[idx].cpu()
            img = img * torch.tensor([0.2023, 0.1994, 0.2010]).view(3, 1, 1) + \
                  torch.tensor([0.4914, 0.4822, 0.4465]).view(3, 1, 1)
            plt.imshow(img.permute(1, 2, 0).clip(0, 1))
            
            # Set title color based on prediction
            pred_class = self.class_names[predictions[idx]]
            if true_labels is not None:
                color = 'green' if predictions[idx] == true_labels[idx] else 'red'
                title = f'Pred: {pred_class}\nTrue: {self.class_names[true_labels[idx]]}'
            else:
                color = 'black'
                title = f'Pred: {pred_class}'
            
            ax1.set_title(title, color=color)
            plt.axis('off')
            
            # Attention heatmap
            ax2 = plt.subplot(num_images, num_cols, idx * num_cols + 2)
            attention_map = avg_attention[idx].cpu()
            sns.heatmap(attention_map, cmap='viridis')
            ax2.set_title('Average Self-Attention')
            
            # Patch-wise attention visualization
            ax3 = plt.subplot(num_images, num_cols, idx * num_cols + 3)
            # Get attention for the classification token (last token)
            patch_attention = attention_map[-1, :-1].reshape(4, 4)  # for 8x8 patches
            sns.heatmap(patch_attention, cmap='viridis')
            ax3.set_title('Patch Attention Weights')
        
        plt.tight_layout()
        return fig

In [ ]:
# Load best model
checkpoint = torch.load('C:\\Users\\Windows\\Documents\\CVC\\repos\seeing-language\\notebooks\wandb\\run-20241111_230849-kjps7qnm\\files\\best_model.pth')
model.load_state_dict(checkpoint['model_state_dict'])

# CIFAR-10 class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# Initialize visualizer
visualizer = GPT2Visualizer(model, device, class_names)

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load CIFAR10
valset = datasets.CIFAR10(root='./data', train=False,
                         download=True, transform=transform_val)

valloader = DataLoader(valset, batch_size=128, shuffle=False, num_workers=2)

# Get some test images
dataiter = iter(valloader)
images, labels = next(dataiter)

# Visualize predictions and attention
fig = visualizer.predict_and_visualize(images[:5], labels[:5])
plt.show()

# To save the figure
# fig.savefig('predictions_attention.png', bbox_inches='tight', dpi=300)